### Atividade -- Data Augmentation

Aluno: Lucas Elias de Andrade Cruvinel

Matricula: 20232011270241

Modelos Utilizados: Rotation e Flips

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shutil
from dataclasses import dataclass
import tensorflow as tf

import keras
from tensorflow.data import Dataset
from keras.models import Sequential, load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, InputLayer, RandomFlip, RandomRotation, Rescaling
from keras.optimizers import RMSprop, Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from livelossplot import PlotLossesKeras
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from keras.utils import image_dataset_from_directory

In [4]:
print(keras.__version__)
print(tf.__version__)

3.2.1
2.16.1


In [2]:
def create_train_test(quant_labels, original_folder="images", root="visaoAugmentation/bd/"):
    subdirs = [name for name in os.listdir(root+original_folder) if os.path.isdir(os.path.join(root+original_folder, name))][:quant_labels]
    for folder in ["train", "test"]:
        if os.path.exists(root + folder):
            shutil.rmtree(root + folder)
        os.mkdir(root + folder)
        for subdir in subdirs:
            os.mkdir(root + folder + "/" + subdir)
    with open(r"visaoAugmentation/bd/meta/train.txt", "r") as f:
        for aux in f.readlines():
            tipo, file_name = aux[:-1].split("/")
            if tipo in subdirs:
                shutil.copy(f"{root}{original_folder}/{tipo}/{file_name}.jpg", f"{root}train/{tipo}/{file_name}.jpg")
    with open(r"visaoAugmentation/bd/meta/test.txt", "r") as f:
        for aux in f.readlines():
            tipo, file_name = aux[:-1].split("/")
            if tipo in subdirs:
                shutil.copy(f"{root}{original_folder}/{tipo}/{file_name}.jpg", f"{root}test/{tipo}/{file_name}.jpg")
    return subdirs
    
quant_labels=10
labels = create_train_test(quant_labels)
print(labels)

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito']


In [ ]:
# trainDS, validDS = image_dataset_from_directory(
#     directory="visaoAugmentation/bd/train",
#     labels="inferred",
#     label_mode="categorical",
#     batch_size=32,
#     image_size=(128, 128),
#     shuffle=True,
#     seed=42,
#     validation_split=0.15,
#     subset="both",
#     interpolation="bilinear",
#     verbose=True,
# )
testDS = image_dataset_from_directory(
    directory="visaoAugmentation/bd/test",
    labels="inferred",
    label_mode="categorical",
    image_size=(128, 128),
    shuffle=False,
    interpolation="bilinear",
    verbose=True,
)

In [ ]:
def create_prof_model(quant_labels, nome="Exemplo",  flip=False, rotation=False):
    model = Sequential(name=nome)

    #### Input Layer ####
    model.add(InputLayer(shape=(128, 128, 3)))
    model.add(Rescaling(1./255))
    if flip:
        model.add(RandomFlip("horizontal_and_vertical"))

    if rotation:
        model.add(RandomRotation(0.2))

    model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', activation='relu'))

    #### Convolutional Layers ####
    model.add(Conv2D(32, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))  # Pooling
    model.add(Dropout(0.2)) # Dropout

    model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
    model.add(Conv2D(128, (3,3), activation='relu'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D((2,2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (5,5), padding='same', activation='relu'))
    model.add(Conv2D(512, (5,5), activation='relu'))
    model.add(MaxPooling2D((4,4)))
    model.add(Dropout(0.2))

    #### Fully-Connected Layer ####
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(quant_labels, activation='softmax'))

    return model
    
model = create_prof_model(quant_labels, flip=True, rotation=True)
model.summary() # a handy way to inspect the architecture

In [ ]:
def run_model(batch_size, n_epochs, optimizer, checkpoint_filepath, nome, flip=False, rotation=False):
    trainDS, validDS = image_dataset_from_directory(
        directory="visaoAugmentation/bd/train",
        labels="inferred",
        label_mode="categorical",
        batch_size=batch_size,
        image_size=(128, 128),
        shuffle=True,
        seed=42,
        validation_split=0.15,
        subset="both",
        interpolation="bilinear",
        verbose=True,
        )

    model = create_prof_model(quant_labels, nome, flip=flip, rotation=rotation)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    # Saves Keras model after each epoch
    checkpointer = ModelCheckpoint(filepath=checkpoint_filepath, verbose=1, save_best_only=True)

    # Early stopping to prevent overtraining and to ensure decreasing validation loss
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')

    history = model.fit(
        trainDS,
        epochs=n_epochs,
        validation_data=validDS,
        callbacks=[
            early_stop, 
            checkpointer, 
            # PlotLossesKeras()
            ],
        verbose=True)
    return history

In [ ]:
@dataclass
class model_data:
    model: str
    labels: list()
    hist: None = None
    y_pred: None = None
    cm: None = None
    report: None = None

    def get_weight_path(self):
        return f"visaoAugmentation/pesos/food_model_{self.model}.weights.keras"

    def execute_model(self, batch_size, n_epochs, flip=False, rotation=False):
        self.hist = run_model(
            batch_size,
            n_epochs,
            optimizer=RMSprop(learning_rate=0.0001), 
            checkpoint_filepath = self.get_weight_path(),
            nome=self.model, 
            flip=flip, 
            rotation=rotation
        )

    def execute_prediction(self, testDS):
        model = load_model(self.get_weight_path())
        predict = model.predict(testDS)
        self.y_pred = np.argmax(predict, axis=-1)
        y_true = np.argmax(np.concatenate([y for x, y in testDS], axis=0), axis=1)
        self.cm = confusion_matrix(y_true, self.y_pred)
        self.report = classification_report(y_true, self.y_pred,target_names=self.labels, output_dict=True, zero_division=0.0)

models_data = {k: model_data(k, labels) for k in ["padrao", "flip", "rotation", "flip_and_rotation"]}

BATCH_SIZE = 32
N_EPOCHS = 2

In [ ]:
models_data["padrao"].execute_model(BATCH_SIZE, N_EPOCHS, flip=False, rotation=False)

In [ ]:
models_data["flip"].execute_model(BATCH_SIZE, N_EPOCHS, flip=True, rotation=False)

In [ ]:
models_data["rotation"].execute_model(BATCH_SIZE, N_EPOCHS, flip=False, rotation=True)

In [ ]:
models_data["flip_and_rotation"].execute_model(BATCH_SIZE, N_EPOCHS, flip=True, rotation=True)

In [ ]:
for k, i in models_data.items():
    print(k)
    i.execute_prediction(testDS)


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 9))

for ind, elem in enumerate(models_data.values()):
    i, j = divmod(ind, 2)
    sns.heatmap(elem.cm, ax=axes[i, j], annot=True, fmt="d", linewidth=.5, cmap="crest", xticklabels=labels, yticklabels=labels)

plt.subplots_adjust(wspace=0.1, hspace=0.4)
plt.savefig("visaoAugmentation/confusion_matrix.png")
plt.plot()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
data = []
for ind, elem in enumerate(models_data.values()):
    data.append([elem.model, "Accuracy ", elem.report["accuracy"]*100.0])
    data.append([elem.model, "Precision ", elem.report["macro avg"]["precision"]*100.0])
    data.append([elem.model, "Recall ", elem.report["macro avg"]["recall"]*100.0])
    data.append([elem.model, "F1-score ", elem.report["macro avg"]["f1-score"]*100.0])

df = pd.DataFrame(data, columns=['Grupo', 'Metrica', 'Porcentagem'])
# ax.grid()
sns.set_style("ticks",{'axes.grid' : True})
sns.barplot(data=df, ax=ax, x='Metrica', y='Porcentagem', hue='Grupo', width=0.6)
plt.savefig("visaoAugmentation/barplot.png")
plt.show()

In [ ]:
for ind, elem in enumerate(models_data.values()):
    fig, axes = plt.subplots(1, 2, figsize=(10, 6))
    axes[0].plot(elem.hist.history['accuracy'])
    axes[0].plot(elem.hist.history['val_accuracy'])
    axes[0].set_title(f'Modelo {elem.model} - Acurácia')
    axes[0].set_ylabel('Acurácia')
    axes[0].set_xlabel('Época')
    axes[0].legend(['Treino', 'Validação'], loc='upper left')

    axes[1].plot(elem.hist.history['loss'])
    axes[1].plot(elem.hist.history['val_loss'])
    axes[1].set_title(f'Modelo {elem.model} - Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Época')
    axes[1].legend(['Treino', 'Validação'], loc='upper left')

    plt.savefig(f"visaoAugmentation/hist_{elem.model}.png")
    plt.show()